In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv("insurance.csv")

df = pd.get_dummies(df, columns=["sex", "smoker", "region"], drop_first=True)

X = df.drop("charges", axis=1).values
y = df["charges"].values.reshape(-1, 1)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def mse_derivative(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size

input_dim = X_train.shape[1]
hidden1 = 64
hidden2 = 32
hidden3 = 16
output_dim = 1

def init_weights(in_dim, out_dim):
    return np.random.randn(in_dim, out_dim) * np.sqrt(2 / in_dim)

W1 = init_weights(input_dim, hidden1)
W2 = init_weights(hidden1, hidden2)
W3 = init_weights(hidden2, hidden3)
W4 = init_weights(hidden3, output_dim)

b1 = np.zeros((1, hidden1))
b2 = np.zeros((1, hidden2))
b3 = np.zeros((1, hidden3))
b4 = np.zeros((1, output_dim))

lr = 0.01
epochs = 100
losses = []

for epoch in range(epochs):
    z1 = X_train @ W1 + b1
    a1 = relu(z1)

    z2 = a1 @ W2 + b2
    a2 = relu(z2)

    z3 = a2 @ W3 + b3
    a3 = relu(z3)

    z4 = a3 @ W4 + b4
    output = z4

    loss = mse_loss(y_train, output)
    losses.append(loss)

    d_loss = mse_derivative(y_train, output)

    dW4 = a3.T @ d_loss
    db4 = np.sum(d_loss, axis=0, keepdims=True)

    dA3 = d_loss @ W4.T * relu_derivative(z3)
    dW3 = a2.T @ dA3
    db3 = np.sum(dA3, axis=0, keepdims=True)

    dA2 = dA3 @ W3.T * relu_derivative(z2)
    dW2 = a1.T @ dA2
    db2 = np.sum(dA2, axis=0, keepdims=True)

    dA1 = dA2 @ W2.T * relu_derivative(z1)
    dW1 = X_train.T @ dA1
    db1 = np.sum(dA1, axis=0, keepdims=True)

    W4 -= lr * dW4
    b4 -= lr * db4

    W3 -= lr * dW3
    b3 -= lr * db3

    W2 -= lr * dW2
    b2 -= lr * db2

    W1 -= lr * dW1
    b1 -= lr * db1

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training Loss Over Epochs")
plt.grid(True)
plt.show()

z1 = X_test @ W1 + b1
a1 = relu(z1)
z2 = a1 @ W2 + b2
a2 = relu(z2)
z3 = a2 @ W3 + b3
a3 = relu(z3)
z4 = a3 @ W4 + b4
y_pred = z4

test_loss = mse_loss(y_test, y_pred)
print(f"Test MSE Loss: {test_loss:.4f}")